Ideia: Dashboard com um input para colocar um emprego ou área e ai baseado nessa entrada, exibiria vários gráficos
mostrando os principais tópicos sobre aquele emrpego, como:
-Salário
-Senioridade
-Localização
-Tipo de vaga (home, hibrido e presencial)
-Principais habilidades
-Benefícios
-Demanda por educação
-Análise de competição: Número de Candidatos por Vaga: Comparação do número de candidatos aplicando para diferentes cargos ou setores.

Listando todas as colunas do dataset

In [59]:
import pandas as pd

jobs = pd.read_csv('./project/app/data/new-dataset/postings.csv')
jobs.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123849 entries, 0 to 123848
Data columns (total 31 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   job_id                      123849 non-null  int64  
 1   company_name                122130 non-null  object 
 2   title                       123849 non-null  object 
 3   description                 123842 non-null  object 
 4   max_salary                  29793 non-null   float64
 5   pay_period                  36073 non-null   object 
 6   location                    123849 non-null  object 
 7   company_id                  122132 non-null  float64
 8   views                       122160 non-null  float64
 9   med_salary                  123849 non-null  float64
 10  min_salary                  29793 non-null   float64
 11  formatted_work_type         123849 non-null  object 
 12  applies                     23320 non-null   float64
 13  original_liste

Exibindo a média dos valores de salário baseadas na senioridade

In [3]:
jobs["title"].head(10)

0                                Marketing Coordinator
1                    Mental Health Therapist/Counselor
2                          Assitant Restaurant Manager
3    Senior Elder Law / Trusts and Estates Associat...
4                                   Service Technician
5             Economic Development and Planning Intern
6                                             Producer
7                                    Building Engineer
8                                Respiratory Therapist
9                                       Worship Leader
Name: title, dtype: object

In [64]:
# Padronizando as colunas max_salary e min_salary para que os valores estejam no período anual
# valor horas * 44 (horas semanais)
# (valor horas semanais) * 4 (quantidade de semanas)
# (valor mensal) * 12 (anual)
jobs[['max_salary', 'min_salary', 'pay_period']] = jobs.apply(
    lambda row:(
        # Caso o pay_period seja 'HOURLY'
        ((row['max_salary'] * 44) * 4) * 12,
        ((row['min_salary'] * 44) * 4) * 12,
        'YEARLY'
    ) if row['pay_period'] == 'HOURLY' else (
        # Caso o pay_period seja 'MONTHLY'
        row['max_salary'] * 12,
        row['min_salary'] * 12,
        'YEARLY'
    ) if row['pay_period'] == 'MONTHLY' else (
        # Caso contrário, mantenha os valores originais
        row['max_salary'],
        row['min_salary'],
        row['pay_period']
    ),
    axis=1,
    result_type='expand'
)
#Criamos esse calculated_med_salary apenas como efeito de comparação das colunas em testes
jobs['calculated_med_salary'] = (jobs['max_salary'] + jobs['min_salary']) / 2
print(jobs[['max_salary', 'min_salary', 'med_salary', 'pay_period', 'calculated_med_salary']].head(10))

   max_salary  min_salary  med_salary pay_period  calculated_med_salary
0     42240.0     35904.0     39072.0     YEARLY                39072.0
1    105600.0     63360.0     84480.0     YEARLY                84480.0
2     65000.0     45000.0     55000.0     YEARLY                55000.0
3    175000.0    140000.0    157500.0     YEARLY               157500.0
4     80000.0     60000.0     70000.0     YEARLY                70000.0
5     42240.0     29568.0     35904.0     YEARLY                35904.0
6    300000.0     60000.0    180000.0     YEARLY               180000.0
7    120000.0     90000.0    105000.0     YEARLY               105000.0
8         NaN         NaN         NaN        NaN                    NaN
9         NaN         NaN       350.0     YEARLY                    NaN


In [78]:
# Os valores da média salarial que estiverem nulos, serem substituidos pela média da soma do max_salary e min_salary
jobs['med_salary'] = jobs.apply(
    lambda row: (row['max_salary'] + row['min_salary']) / 2 
    if row['med_salary'] <= 0 else row['med_salary'],
    axis=1
)
jobs['med_salary'] = jobs['med_salary'].round(-3)
print(jobs[['max_salary', 'min_salary', 'med_salary', 'calculated_med_salary']].head(10))

   max_salary  min_salary  med_salary  calculated_med_salary
0     42240.0     35904.0     39000.0                39072.0
1    105600.0     63360.0     84000.0                84480.0
2     65000.0     45000.0     55000.0                55000.0
3    175000.0    140000.0    158000.0               157500.0
4     80000.0     60000.0     70000.0                70000.0
5     42240.0     29568.0     36000.0                35904.0
6    300000.0     60000.0    180000.0               180000.0
7    120000.0     90000.0    105000.0               105000.0
8         NaN         NaN         NaN                    NaN
9         NaN         NaN         NaN                    NaN


In [79]:
# Agrupando o salário de acordo com a senioridade
jobs[["formatted_experience_level", "med_salary"]].groupby("formatted_experience_level").median().reset_index()

,formatted_experience_level,med_salary
0,Associate,75000.0
1,Director,168000.0
2,Entry level,58000.0
3,Executive,195000.0
4,Internship,52500.0
5,Mid-Senior level,110000.0


In [80]:
#Escrevendo um gráfico em um html mostrando a proporção Salário x Senioridade das vagas
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go

median_salaries=jobs[["formatted_experience_level", "med_salary"]].groupby("formatted_experience_level").median().reset_index()
fig = go.Figure(
        data=go.Bar(x=median_salaries["formatted_experience_level"],
                    y=median_salaries["med_salary"],
                    text=median_salaries["med_salary"],
                    texttemplate='%{text:.2f}',  
                    textposition='outside'
                    ))
fig.write_html('first_figure.html', auto_open=True)


Um gráfico que seria baseado em um select listando algumas profissões, quando escolher uma, vai mostrar a proporção salário x senioridade daquela profissão em específico.
TA QUEBRADO:

In [82]:
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
cargos = []
# median_salaries=jobs[["formatted_experience_level", "max_salary"]].groupby("formatted_experience_level").median().reset_index()

app = Dash(__name__)

app.layout = html.Div([
    html.H4('Análise da média salarial x senioridade'),
    html.P("Selecione o cargo:"),
    dcc.Dropdown(
        id="dropdown",
        options=["Leader", "Assitant", "Producer", "Developer"],
        value='Leader',
        clearable=False,
    ),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output("graph", "figure"), 
    Input("dropdown", "value"))
def display_cargo(cargo):
    filtered_jobs = jobs[jobs["title"].str.contains(cargo, case=False, na=False)]
    median_salaries=filtered_jobs[["formatted_experience_level", "med_salary"]].groupby("formatted_experience_level").median().reset_index()
    fig = go.Figure(
        data=go.Bar(x=median_salaries["formatted_experience_level"],
                    y=median_salaries["med_salary"],
                    textposition='outside'))
    return fig


app.run_server(debug=True)

Distribuição salarial 
A distribuição salarial mostra que a maioria dos empregos está concentrada nas faixas salariais mais baixas, com alguns valores discrepantes com salários elevados.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv("project/linkedin_dashboard/data/postings.csv")  

# Distribuição salarial
plt.figure(figsize=(10, 6))
sns.histplot(data=df[df['normalized_salary'] < 500000], x='normalized_salary', bins=50)
plt.title('Distribuição de Salários Normalizados (< $500k)')
plt.xlabel('Salário Normalizado ($)')
plt.ylabel('Contagem')
plt.show()

# Top 10 títulos
plt.figure(figsize=(12, 6))
df['title'].value_counts().head(10).plot(kind='bar')
plt.title('Top 10 Títulos Mais Comuns de Vagas')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


Distribuição geógrafica 

In [ ]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv("project/linkedin_dashboard/data/postings.csv")  

#cargos de interesse
cargos_de_interesse = ['Data Scientist', 'Software Engineer', 'Product Manager'] 
df_filtrado = df[df['title'].isin(cargos_de_interesse)]  

plt.figure(figsize=(14, 8))
contagem_locacoes = df_filtrado['location'].value_counts().head(20)
contagem_locacoes.plot(kind='bar')
plt.title('Top 20 Localizações para Cargos Selecionados')
plt.xlabel('Localização')
plt.ylabel('Número de Vagas')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()